In [1]:
from IPython.display import HTML

import math
import pandas as pd
import numpy as np

#from scipy.optimize import minimize
#from scipy.optimize import differential_evolution    

from lmfit import Minimizer, Parameters, report_fit

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import LinearAxis, Range1d
from bokeh.models import DatetimeTickFormatter, MonthsTicker, NumeralTickFormatter, Legend

output_notebook()

Loading BokehJS ...

In [2]:
import sys
sys.path.insert(1,'../covid-website')

import SISV as s
from data import Data
from ContactRate import contact_rate
from SISV_calib import SISV_lmfit


In [11]:
source = 'Johns Hopkins'    
region = 'US'
state = 'New York'

cutoff_positive = 1
cutoff_death = 1
truncate = 0
window = 14

d = Data(source=source, region=region, state=state, county="", cutoff_positive=cutoff_positive, cutoff_death=cutoff_death, truncate=truncate) 


In [27]:

#https://www.cdc.gov/coronavirus/2019-ncov/hcp/planning-scenarios.html
#exposure to symptom = 6 days
#symptom onset to symptom onset 6 days
#detection rate 11%
#symptom onset to test 3 days
#symptom to hospitalization 6 days (<65y olds)   4 days (>=65y)
#hospital stay if not ICU 4 days
#hospital stay if ICU 14 days
#ICU percentage 36% of admitted
#mechanical ventilation 22% (ICU and not ICU)
#length of mechanical ventilation 6 days
#fatalities of hospitalized  2.4% (18-49y) 10% (50-64y) and 27% ()>=65)
#symptom to death 15 days (18-49y) 17 days (50-64y) 13 days (>=65y)
#death to reporting 21 days

#missing from CDC: testing to reporting


#IFR by age group  population weighted average = 0.73%
#0-19 years: 0.00003   82million	
#20-49 years: 0.0002   130million
#50-69 years: 0.005    80million
#70+ years: 0.054      37million
#population: https://www.census.gov/data/tables/time-series/demo/popest/2010s-national-detail.html#par_textimage_1537638156


p1 = SISV_lmfit(d, solver='leastsq',
                  overrides={
                      'exp_stages'  : 0,
                      'inf_stages'  : 1,
                      'crit_stages' : 5,
                      'test_stages' : 3,
                      'death_rate'  : 0.73e-2,
                      'gamma_exp'   : 1/4,
                      'gamma'       : 1/4,
                      'gamma_crit'  : 1/(17+21),
                      'gamma_pos'   : 1/(5+10),
                      'interv'      :'piecewise linear',
                      'init_beta'   : '',
                      'minwindow'   : 7,
                      'segments'    : 8,
                  })

y1 = s.SISV(d.x, p1) 




In [28]:
print('S:', y1[-1, s.cS] )
print('I:', y1[-1, s.cI] )
print('C:', y1[-1, s.cC] )
print(1/1.7, y1[-1, s.cS]/d.population )

S: -105.85036731848923
I: 2.2339762720346423e-34
C: 2.474425516588393e-15
0.5882352941176471 -5.441182070392625e-06


In [29]:
x = np.arange(700)
y2 = s.SISV(x, p1) 
p = figure(title='Excess Fatalities', plot_width=800, plot_height=600 , y_axis_type="log")
p.y_range.start = 1
p.line(d.x[d.minD+1:], d.dfatalities, line_width=1, line_color='red', line_dash='dotted', alpha=0.3)
p.line(x[1:], np.diff(y2[:,s.cF]), line_width=1, line_color='black', line_dash='solid', alpha=0.7)

p.line(d.x[d.minP+1:], d.dpositives, line_width=1, line_color='red', line_dash='dotted', alpha=0.3)
p.line(x[1:], np.diff(y2[:,s.cP]), line_width=1, line_color='black', line_dash='solid', alpha=0.7)

show(p)

In [30]:


def print_params(title, params):
    print('----------')
    print(title)
    print("detect:", params["detection_rate"])
    print("i0:", params["i0"])
    print("R0:", params['beta0']/params['gamma'])
    for i in range(1, 1+params['segments']):
          print(params['t{}'.format(i)],":", params['beta{}'.format(i)]/params['gamma'])

print_params("p1", p1)
#print_params("p2", p2)
    
p = figure(title='Excess Fatalities', plot_width=800, plot_height=600 , y_axis_type="log")
p.y_range.start = 1

r0 = p.line(d.xd[d.minD+1:], d.dfatalities, line_width=1, line_color='red', line_dash='dotted', alpha=0.3)
r1 = p.circle(d.xd[d.minD+1:], d.dfatalities, size=5, color="red", alpha=0.3)

#plot 7-day rolling average
rolling = pd.DataFrame(data = d.dfatalities).interpolate().rolling(7).mean()
r2 = p.line(d.xd[d.minD+1:], rolling.loc[:,0].values, line_width=1, line_color='red')

r3_1 = p.line(d.xd[d.minD+1:], np.diff(y1[d.minD:,s.cF]), line_width=1, line_color='black', line_dash='solid', alpha=0.7)

#-------------

r0_p = p.line(d.xd[d.minP+1:], d.dpositives, line_width=1, line_color='red', line_dash='dotted', alpha=0.3)
r1_p = p.circle(d.xd[d.minP+1:], d.dpositives, size=5, color="red", alpha=0.3)

r3_1_p = p.line(d.xd[d.minP+1:], np.diff(y1[d.minP:,s.cP]), line_width=1, line_color='black', line_dash='solid', alpha=0.7)

legend = [
    ("COVID fatalities"   , [r0, r1]),
    ("COVID 7-day average"   , [r2]),
    
    ("p1", [r3_1]),

    ("COVID positives"   , [r0_p, r1_p]),
    ("p1_p", [r3_1_p]),
]

    
p.add_layout(Legend(items=legend, location='center'), 'right')
   
p.yaxis[0].formatter = NumeralTickFormatter(format="0,0")

p.xaxis.ticker = MonthsTicker(months=list(range(1,13)))

p.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

p.xgrid.ticker = p.xaxis.ticker

p.xaxis.major_label_orientation = math.pi/4


p.extra_x_ranges['x2'] = Range1d(d.x[0], d.x[-1])
ax2 = LinearAxis(x_range_name="x2", axis_label="days")
p.add_layout(ax2, 'below')


p.ygrid.grid_line_color = 'navy'
p.ygrid.grid_line_alpha = 0.3
p.ygrid.minor_grid_line_color = 'navy'
p.ygrid.minor_grid_line_alpha = 0.1

p.yaxis[0].axis_label = 'Number of deaths per day'

#p.toolbar_location = None
show(p)

rplot = figure(title='R0')
R0_1 = contact_rate(d.x, p1) / p1['gamma']
rplot.line(d.x, R0_1, line_width=1, line_color='black', line_dash='solid', alpha=1)
show(rplot)


----------
p1
detect: 0.027302512923233102
i0: 407892.80976785254
R0: 9.99890548939512
141.0 : 5.087185880546506
215.00000002114672 : 5.087185880546506
258.318366550467 : 5.087185880546506
280.66813343423416 : 6.443323611742074
295.78006803806363 : 9.700507862954112
306.3900363945552 : 5.286072920942472
315.24668903231435 : 6.447761185917898
323.5612303586037 : 6.86101941220448


In [31]:
p = figure(title='Cumul', plot_width=800, plot_height=600 , y_axis_type="log")
p.y_range.start = 1

r0 = p.line(d.xd[d.minD:], d.fatalities[d.minD:], line_width=3, line_color='red', line_dash='solid', alpha=1)
r3_1 = p.line(d.xd[d.minD:], y1[d.minD:,s.cF], line_width=1, line_color='black', line_dash='solid', alpha=1)

#-------------

r0_p = p.line(d.xd[d.minP:], d.positives[d.minP:], line_width=3, line_color='red', line_dash='solid', alpha=1)
r3_1_p = p.line(d.xd[d.minP:], y1[d.minP:,s.cP], line_width=1, line_color='black', line_dash='solid', alpha=1)
    
p.yaxis[0].formatter = NumeralTickFormatter(format="0,0")
p.xaxis.ticker = MonthsTicker(months=list(range(1,13)))
p.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

p.xgrid.ticker = p.xaxis.ticker
p.xaxis.major_label_orientation = math.pi/4

p.extra_x_ranges['x2'] = Range1d(d.x[0], d.x[-1])
ax2 = LinearAxis(x_range_name="x2", axis_label="days")
p.add_layout(ax2, 'below')


p.ygrid.grid_line_color = 'navy'
p.ygrid.grid_line_alpha = 0.3
p.ygrid.minor_grid_line_color = 'navy'
p.ygrid.minor_grid_line_alpha = 0.1

p.yaxis[0].axis_label = 'Number of deaths per day'

show(p)